In [1]:
import torch
from torch.nn.functional import softmax
from shared import (
    gpt,
    show_probabilities,
    tokenizer,
)

### P(u)

![gpt1math](assets/pumath.png)

In [2]:
vocab = ['a', 'b', 'c', 'd', 'e']
logits = torch.tensor([2.5, 1.0, 0.5, -1.0, 3.0])
probabilities = softmax(logits, dim=0)

show_probabilities(logits, probabilities, vocab)

┏━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┓
┃ Token ┃ Logit ┃ Probability ┃
┣━━━━━━━╋━━━━━━━╋━━━━━━━━━━━━━┫
┃ 'a'   ┃  2.50 ┃      0.3292 ┃
┃ 'b'   ┃  1.00 ┃      0.0735 ┃
┃ 'c'   ┃  0.50 ┃      0.0446 ┃
┃ 'd'   ┃ -1.00 ┃      0.0099 ┃
┃ 'e'   ┃  3.00 ┃      0.5428 ┃
┗━━━━━━━┻━━━━━━━┻━━━━━━━━━━━━━┛

In [3]:
text = 'The sun rises in the'
inputs = tokenizer(text, return_tensors='pt')

with torch.no_grad():
    outputs = gpt(**inputs)

last_token_logits = outputs.logits[0, -1, :]

probabilities = softmax(last_token_logits, dim=0)

# get top 10 most likely next tokens
top_probs, top_indices = probabilities.topk(10)
top_logits = last_token_logits[top_indices]
top_tokens = [tokenizer.decode([idx]) for idx in top_indices]

show_probabilities(top_logits, top_probs, top_tokens)

┏━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━┓
┃    Token    ┃ Logit ┃ Probability ┃
┣━━━━━━━━━━━━━╋━━━━━━━╋━━━━━━━━━━━━━┫
┃ 'east'      ┃  7.68 ┃      0.2897 ┃
┃ 'morning'   ┃  7.43 ┃      0.2243 ┃
┃ 'west'      ┃  7.22 ┃      0.1814 ┃
┃ 'sky'       ┃  6.64 ┃      0.1018 ┃
┃ 'north'     ┃  4.59 ┃      0.0132 ┃
┃ 'south'     ┃  4.26 ┃      0.0095 ┃
┃ 'middle'    ┃  4.23 ┃      0.0092 ┃
┃ 'afternoon' ┃  4.15 ┃      0.0084 ┃
┃ 'distance'  ┃  3.99 ┃      0.0072 ┃
┃ 'early'     ┃  3.98 ┃      0.0071 ┃
┗━━━━━━━━━━━━━┻━━━━━━━┻━━━━━━━━━━━━━┛

#### Sampling

Once we have probabilities, how do we actually pick the next word?

- **Greedy sampling**: Always pick the highest probability word (deterministic)
- **Top-k**: Sample from the k most likely tokens when sampling
- **Top-p (nucleus)**: Sample from smallest set of tokens whose cumulative probability >= p
- **Beam search**: Heuristic search algorithm that explores multiple possible sequences simultaneously by keeping the top-k most promising partial sequences (called "beams") at each step, where k is the beam width
- **Structured Generation**: Only consider certain parts of the vocabulary as valid
- **Temperature**: Scale logits before softmax to control randomness (higher = more creative)

Different sampling strategies produce different text styles.

In [4]:
text = 'thanks for coming out tonight!'
inputs = tokenizer(text, return_tensors='pt')
generated = gpt.generate(inputs.input_ids, max_length=20, do_sample=True, temperature=0.7)
print(tokenizer.decode(generated[0], skip_special_tokens=True))

thanks for coming out tonight ! " i said . " i have a ton of stuff to do .
